## Pandas

Материалы:
* Макрушин С.В. "Лекция 2: Библиотека Pandas"
* https://pandas.pydata.org/docs/user_guide/index.html#
* https://pandas.pydata.org/docs/reference/index.html
* Уэс Маккини. Python и анализ данных

In [1]:
import pandas as pd
import numpy as np

## Задачи для совместного разбора

1. Загрузите данные из файла `sp500hst.txt` и обозначьте столбцы в соответствии с содержимым: `"date", "ticker", "open", "high", "low", "close", "volume"`.

In [2]:
dataframe = pd.read_csv('files/sp500hst.txt', delimiter=',', names=["date", "ticker", "open", "high", "low", "close", "volume"])
dataframe.head()

FileNotFoundError: [Errno 2] No such file or directory: 'files/sp500hst.txt'

2. Рассчитайте среднее значение показателей для каждого из столбцов c номерами 3-6.

In [ ]:
dataframe.iloc[:, 3:7].mean(axis=0)

3. Добавьте столбец, содержащий только число месяца, к которому относится дата.

In [ ]:
dataframe['mounth'] = dataframe.date//100%100
dataframe

4. Рассчитайте суммарный объем торгов для для одинаковых значений тикеров.

In [ ]:
dataframe.groupby('ticker').volume.sum()# .loc[:, 'volume']

5. Загрузите данные из файла sp500hst.txt и обозначьте столбцы в соответствии с содержимым: "date", "ticker", "open", "high", "low", "close", "volume". Добавьте столбец с расшифровкой названия тикера, используя данные из файла `sp_data2.csv` . В случае нехватки данных об именах тикеров корректно обработать их.

In [ ]:
tickers = pd.read_csv('files/sp_data2.csv', delimiter=';', header=None, names=["ticker", 'name', 'persent'])
dataframe.merge(tickers)

## Лабораторная работа №2

### Базовые операции с `DataFrame`

1.1 В файлах `recipes_sample.csv` и `reviews_sample.csv` находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца с индексами в таблице `reviews` (безымянный столбец).

In [ ]:
reviews = pd.read_csv('files/reviews_sample.csv', delimiter=',', index_col=0, parse_dates=['date'])
reviews

In [ ]:
reviews

1.2 Для каждой из таблиц выведите основные параметры:
* количество точек данных (строк);
* количество столбцов;
* тип данных каждого столбца.

In [ ]:
recipes.dtypes, recipes.shape

In [ ]:
reviews.dtypes, reviews.shape

1.3 Исследуйте, в каких столбцах таблиц содержатся пропуски. Посчитайте долю строк, содержащих пропуски, в отношении к общему количеству строк.

In [ ]:
rows_with_missing_values = recipes.isnull().any(axis=1).sum()
str(rows_with_missing_values/recipes.shape[0]*100) + '%'

In [ ]:
rows_with_missing_values = reviews.isnull().any(axis=1).sum()
str(rows_with_missing_values/reviews.shape[0]*100) + '%'

1.4 Рассчитайте среднее значение для каждого из числовых столбцов (где это имеет смысл).

In [ ]:
recipes.loc[:, ['minutes', 'n_ingredients']].mean(), reviews.loc[:, ['rating']].mean()

1.5 Создайте серию из 10 случайных названий рецептов.

In [ ]:
names = recipes.name.sample(10)
names, type(names)

1.6 Измените индекс в таблице `reviews`, пронумеровав строки, начиная с нуля.

In [ ]:
reviews.index = range(0, reviews.shape[0])
reviews

1.7 Выведите информацию о рецептах, время выполнения которых не больше 20 минут и кол-во ингредиентов в которых не больше 5.

In [ ]:
recipes[(recipes.minutes <= 20) & (recipes.n_ingredients <=5)]

### Работа с датами в `pandas`

2.1 Преобразуйте столбец `submitted` из таблицы `recipes` в формат времени. Модифицируйте решение задачи 1.1 так, чтобы считать столбец сразу в нужном формате.

In [ ]:
recipes = pd.read_csv('files/recipes_sample.csv', delimiter=',', parse_dates=['submitted'])
recipes

2.2 Выведите информацию о рецептах, добавленных в датасет не позже 2010 года.

In [ ]:
recipes[(recipes.submitted < '2011.01.01')]

### Работа со строковыми данными в `pandas`

3.1  Добавьте в таблицу `recipes` столбец `description_length`, в котором хранится длина описания рецепта из столбца `description`.

In [ ]:
recipes['description_length'] = recipes.description.apply(lambda x: len(str(x)))
recipes.head()

3.2 Измените название каждого рецепта в таблице `recipes` таким образом, чтобы каждое слово в названии начиналось с прописной буквы.

In [ ]:
a = 'hello world'
a.capitalize()

In [ ]:
recipes.name = recipes.name.apply(lambda x: ' '.join(word.capitalize() for word in x.split()))
recipes

3.3 Добавьте в таблицу `recipes` столбец `name_word_count`, в котором хранится количество слов из названии рецепта (считайте, что слова в названии разделяются только пробелами). Обратите внимание, что между словами может располагаться несколько пробелов подряд.

In [ ]:
recipes['name_word_count'] = recipes.name.apply(lambda x: len(x.split()))
recipes.head()

### Группировки таблиц `pd.DataFrame`

4.1 Посчитайте количество рецептов, представленных каждым из участников (`contributor_id`). Какой участник добавил максимальное кол-во рецептов?

In [ ]:
recipes_founder_count = recipes.groupby('contributor_id').size().reset_index(name='recipe_count')
recipes_founder_count

In [ ]:
max_recipes_founder = recipes_founder_count.loc[recipes_founder_count['recipe_count'].idxmax()]
max_recipes_founder

4.2 Посчитайте средний рейтинг к каждому из рецептов. Для скольких рецептов отсутствуют отзывы? Обратите внимание, что отзыв с нулевым рейтингом или не заполненным текстовым описанием не считается отсутствующим.

Посчитайте средний рейтинг к каждому из рецептов

In [ ]:
marks = reviews.groupby('recipe_id').rating.mean()
marks

Для скольких рецептов отсутствуют отзывы?

In [ ]:
recipes.shape[0] - pd.unique(reviews.recipe_id).shape[0] 

In [ ]:
recipes.shape[0] - marks.shape[0]

4.3 Посчитайте количество рецептов с разбивкой по годам создания.

In [ ]:
recipes.groupby(recipes['submitted'].dt.year).size()

### Объединение таблиц `pd.DataFrame`

In [ ]:
recipes.columns, reviews.columns

5.1 При помощи объединения таблиц, создайте `DataFrame`, состоящий из четырех столбцов: `id`, `name`, `user_id`, `rating`. Рецепты, на которые не оставлен ни один отзыв, должны отсутствовать в полученной таблице. Подтвердите правильность работы вашего кода, выбрав рецепт, не имеющий отзывов, и попытавшись найти строку, соответствующую этому рецепту, в полученном `DataFrame`.

In [ ]:
# recipes.merge(reviews, left_on='id', right_on='recipe_id').loc[:, 'rating']
merged_df = recipes.merge(reviews, left_on='id', right_on='recipe_id').loc[:, ['id', 'name', 'user_id', 'rating']]
merged_df.head()

In [ ]:
# 1. Объединяем таблицы по столбцам 'id' и 'recipe_id'
merged_df = pd.merge(recipes, reviews, left_on='id', right_on='recipe_id')

# 2. Отбираем только те строки, где есть отзывы
merged_df = merged_df.dropna(subset=['rating'])

# 3. Выбираем нужные столбцы
result_df = merged_df[['id', 'name', 'user_id', 'rating']]
result_df

5.2 При помощи объединения таблиц и группировок, создайте `DataFrame`, состоящий из трех столбцов: `recipe_id`, `name`, `review_count`, где столбец `review_count` содержит кол-во отзывов, оставленных на рецепт `recipe_id`. У рецептов, на которые не оставлен ни один отзыв, в столбце `review_count` должен быть указан 0. Подтвердите правильность работы вашего кода, выбрав рецепт, не имеющий отзывов, и найдя строку, соответствующую этому рецепту, в полученном `DataFrame`.

In [ ]:
merged_df['review_count'] = merged_df.groupby('id').count()['rating']
merged_df = merged_df.dropna(subset=['review_count']).loc[:, ['id', 'name', 'review_count']].rename(columns={'id': 'recipes_id'})
merged_df.head()

5.3. Выясните, рецепты, добавленные в каком году, имеют наименьший средний рейтинг?

In [ ]:
avg_rating_per_year = reviews.groupby(reviews['date'].dt.year)['rating'].mean()
avg_rating_per_year[avg_rating_per_year < avg_rating_per_year.quantile(q=0.2)]

### Сохранение таблиц `pd.DataFrame`

6.1 Отсортируйте таблицу в порядке убывания величины столбца `name_word_count` и сохраните результаты выполнения заданий 3.1-3.3 в csv файл. 

In [ ]:
recipes.sort_values(['name_word_count']).to_csv('files/result_recipes.csv')
reviews.to_csv('files/result_reviews.csv')

6.2 Воспользовавшись `pd.ExcelWriter`, cохраните результаты 5.1 и 5.2 в файл: на лист с названием `Рецепты с оценками` сохраните результаты выполнения 5.1; на лист с названием `Количество отзывов по рецептам` сохраните результаты выполнения 5.2.

In [ ]:
with pd.ExcelWriter('files/Рецепты с оценками.xlsx') as file:
    merged_df.to_excel(file)